In [2]:
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
# Mount Google Drive to access your dataset if needed
from google.colab import drive
drive.mount('/content/drive')

# Define dataset paths
dataset_path = '/content/drive/MyDrive/Nail Images/train'
test_dataset_path = '/content/drive/MyDrive/Nail Images/test'

# Step 1: Load Datasets using image_dataset_from_directory
batch_size = 8
img_size = (224, 224)

# Load the training and validation datasets
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    image_size=img_size,
    batch_size=batch_size,
    label_mode='categorical',
    validation_split=0.2,
    subset='training',
    seed=42,
)

val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    dataset_path,
    image_size=img_size,
    batch_size=batch_size,
    label_mode='categorical',
    validation_split=0.2,
    subset='validation',
    seed=42,
)

# Load the test dataset
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    test_dataset_path,
    image_size=img_size,
    batch_size=batch_size,
    label_mode='categorical',
)

# Retrieve class names
class_names = train_dataset.class_names
print(f"Class names: {class_names}")

# Step 2: Normalize the datasets
def normalize_img(image, label):
    return tf.cast(image, tf.float32) / 255.0, label

train_dataset = train_dataset.map(normalize_img)
val_dataset = val_dataset.map(normalize_img)
test_dataset = test_dataset.map(normalize_img)

# Step 3: Load Pre-Trained Model (MobileNetV2)
base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation="relu")(x)  # Reduced size for small dataset
x = Dropout(0.4)(x)  # Add dropout for regularization
predictions = Dense(len(class_names), activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers initially
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss="categorical_crossentropy", metrics=["accuracy"])

# Step 4: Train the Model
epochs = 64  # Reduced epochs for smaller dataset

history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=epochs,
    verbose=1,
)

# Step 5: Evaluate the Model
loss, accuracy = model.evaluate(test_dataset, verbose=1)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

# Step 6: Save the Model
# Save the model in the SavedModel format
saved_model_path = "/content/drive/MyDrive/nail_disease_saved_model"
model.export(saved_model_path)
print(f"Model saved in SavedModel format at {saved_model_path}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Found 661 files belonging to 17 classes.
Using 529 files for training.
Found 661 files belonging to 17 classes.
Using 132 files for validation.
Found 183 files belonging to 17 classes.
Class names: ['Darier_s disease', 'Muehrck-e_s lines', 'aloperia areata', 'beau_s lines', 'bluish nail', 'clubbing', 'eczema', 'half and half nailes (Lindsay_s nails)', 'koilonychia', 'leukonychia', 'onycholycis', 'pale nail', 'red lunula', 'splinter hemmorrage', 'terry_s nail', 'white nail', 'yellow nails']
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/64
67/67 ━━━━━━━━━━━━━━━━━━━━ 195s 3s/step - accuracy: 0.0976 - loss: 3.0716 - val_accuracy: 0.2500 - val_loss: 2.5717
Epoch 2/64
67/67 ━━━━━━━━━━━━━━━━━━━━ 16s 56ms/step - accuracy: 0.2235 - loss: 2.4928 - val_accuracy: 0.3485 - val_loss: 2.3208
Epoch 3/64
67/67 ━━━━━━━━━━━━━━━━━━━━ 3s 45ms/step - accuracy: 0.3061 - 